In [2]:
import pandas as pd

In [25]:
patient_registry = pd.read_csv('data/patient_registry.csv')

In [26]:
patient_registry.head()

,patient_id,age_group,risk_segment
0,P_0000,36-50,High
1,P_0001,50-65,Medium
2,P_0002,18-35,Low
3,P_0003,18-35,High
4,P_0004,50-65,Medium


In [27]:
patient_registry.info()

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   patient_id    500 non-null    str  
 1   age_group     500 non-null    str  
 2   risk_segment  500 non-null    str  
dtypes: str(3)
memory usage: 11.8 KB


In [29]:
app_logs = pd.read_json('data/app_logs.jsonl', lines=True)

In [30]:
app_logs.head()

,event_id,patient_id,timestamp,event_type,payload
0,evt_898,P_0023,2023-11-14 19:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}
1,evt_9843,P_0256,2023-10-18 13:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}
2,evt_15683,P_0407,2023-11-05 19:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
3,evt_5567,P_0147,2023-10-17 11:09:00,measurement_logged,{'glucose_value': 95}
4,evt_8135,P_0212,2023-11-07 11:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}


In [31]:
app_logs.info()

<class 'pandas.DataFrame'>
RangeIndex: 19176 entries, 0 to 19175
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_id    19176 non-null  str           
 1   patient_id  19176 non-null  str           
 2   timestamp   19176 non-null  datetime64[us]
 3   event_type  19176 non-null  str           
 4   payload     19176 non-null  object        
dtypes: datetime64[us](1), object(1), str(3)
memory usage: 749.2+ KB


In [21]:
data.event_type.value_counts()

event_type
nudge_sent            13590
measurement_logged     5586
Name: count, dtype: int64

In [24]:
data[data['event_type'] == 'nudge_sent']

,event_id,patient_id,timestamp,event_type,payload
0,evt_898,P_0023,2023-11-14 19:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}
1,evt_9843,P_0256,2023-10-18 13:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}
2,evt_15683,P_0407,2023-11-05 19:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
4,evt_8135,P_0212,2023-11-07 11:00:00,nudge_sent,{'nudge_type': 'Gentle_Reminder'}
6,evt_9306,P_0242,2023-10-14 18:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
...,...,...,...,...,...
19168,evt_11471,P_0298,2023-10-02 19:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
19170,evt_17860,P_0466,2023-10-03 16:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
19171,evt_6847,P_0180,2023-11-03 18:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
19172,evt_12284,P_0320,2023-10-17 17:00:00,nudge_sent,{'nudge_type': 'Urgent_Alert'}
